In [ ]:
# =============================================================================
# Step 1: Environment Setup - Mount Drive and Install Dependencies
# =============================================================================
from google.colab import drive
drive.mount('/content/drive')

# Install required packages
!pip install torch pandas tqdm scikit-learn
!pip install torch_geometric
!pip install -e /content/drive/MyDrive/PhD/Study2_review/Published_Codes_Study_2/Production/codes
!pip install accelerate
!pip install -U bitsandbytes
!pip install peft

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.1 MB/s eta 0:00:00
Obtaining file:///content/drive/MyDrive/PhD/Study2_review/Published_Codes_Study_2/Production/codes
  Preparing metadata (setup.py) ... done
  Running setup.py develop for codes
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 37.1 MB/s eta 0:00:00


In [ ]:
# =============================================================================
# Step 2: Imports, Deterministic Setup, BERT Model & Tokenizer
# =============================================================================
import os
import warnings
warnings.filterwarnings("ignore")

import random
import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer
import pandas as pd
from tqdm import tqdm
import sqlite3
import json

tqdm.pandas()

# -------- Deterministic setup --------
SEED = 9898
os.environ["PYTHONHASHSEED"] = str(SEED)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"  # helps determinism on CUDA

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# If some ops are not deterministic, warn_only=True avoids hard crashes.
torch.use_deterministic_algorithms(True, warn_only=True)

# -------- Device --------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -------- BERT model & tokenizer (FP32/FP16, NOT 8-bit) --------
model_name = "google-bert/bert-large-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModel.from_pretrained(model_name)
# Optional: use half precision if VRAM is tight
# model = model.half()
model.to(device)
model.eval()  # VERY important: turns off dropout inside BERT

# =============================================================================
# Step 3: Deterministic BERT Encoding Function
# =============================================================================
def bertEncode(inputs, max_len=100, batch_size=32):
    """
    Encodes a list of text inputs into BERT embeddings using mini-batches.

    Args:
        inputs (list of str): Text data to encode.
        max_len (int): Maximum sequence length for encoding.
        batch_size (int): Number of texts to process at once.

    Returns:
        list: List of embeddings (each embedding is a 1D numpy array).
    """
    all_embs = []

    # Deterministic batching over a fixed range
    for start in range(0, len(inputs), batch_size):
        batch_texts = inputs[start:start + batch_size]

        encoding = tokenizer(
            batch_texts,
            truncation=True,
            return_tensors="pt",
            add_special_tokens=True,
            max_length=max_len,
            padding="max_length",
        )

        with torch.no_grad():
            outputs = model(
                input_ids=encoding["input_ids"].to(device),
                attention_mask=encoding["attention_mask"].to(device),
            )
            # More stable than mean-pooling: CLS token
            batch_embs = outputs.last_hidden_state[:, 0, :]  # [batch, hidden]

        all_embs.extend(batch_embs.cpu().numpy())

    return all_embs


Using device: cuda


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [ ]:
import re

def compact_tesa(raw: str) -> str:
    """
    Convert verbose TESA like:
       'neutral emotion & AGAINST stance towards Warren; ...'
    into grouped compact tags:
       '[Emotion:neutral Stance:AGAINST Target:Warren]'
    """
    if pd.isna(raw) or not isinstance(raw, str):
        return ""

    segments = [s.strip(" ,") for s in raw.split(";") if s.strip(" ,")]

    tags = []
    for seg in segments:
        try:
            # Extract emotion
            emo = seg.split(" emotion")[0].strip()

            # Extract stance
            after_amp = seg.split("&", 1)[1]
            stance = after_amp.split(" stance")[0].strip()

            # Extract target
            target = seg.split("towards", 1)[1].strip(" ,")
            target = target.replace(" ", "_")
            target = re.sub(r"[^A-Za-z0-9_]", "", target)

            tags.append(f"[Emotion:{emo} Stance:{stance} Target:{target}]")
        except:
            continue

    return " ".join(tags)




In [ ]:
#checking if 500 max token is ok
import sqlite3
import pandas as pd
from transformers import AutoTokenizer


# =============================================================================
# Step 4: Set Up Dataset Paths and Configurations
# =============================================================================
base_path = '/content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/Production/data'

# Define dataset paths and their input mode configurations (each mode has a max length setting)
datasets = {
    f'{base_path}/fibvid': {'text_emo': 500, 'text_only': 500},
    f'{base_path}/ts':     {'text_emo': 500, 'text_only': 500},
}

# =============================================================================
# Step 5: Process Datasets and Generate BERT Embeddings into Separate Databases
# =============================================================================
for dataset_path, mode_config in datasets.items():
    db_path = f'{dataset_path}_data.db'
    print(f"\n==============================")
    print(f"Processing database: {db_path}")
    con = sqlite3.connect(db_path)

    for input_mode, max_len in mode_config.items():
        print(f"\n----- Mode: {input_mode} (max_len={max_len}) -----")

        # --- Load data deterministically ---
        data = pd.read_sql_query('SELECT * FROM data', con)

        print(f"\nRaw data preview for {dataset_path} ({input_mode}):")
        display(data.head())
        print("Example raw text:")
        display(data.text.iloc[0])

        # --- Build the input text used for BERT ---
        if input_mode == 'text_emo':
            # Emotion + stance + original text
            #data['input'] = data['target_emotion_stance'].fillna('') + '[SEP]' + data['text'].fillna('')
            data['tesa_tags'] = data['target_emotion_stance'].fillna("").apply(compact_tesa)
            data['input'] = data['tesa_tags'] + " [SEP] " + data['text'].fillna("")

        else:
            # Text only
            data['input'] = data['text'].fillna('')

        # --- NEW: compute token lengths for the whole dataset (no truncation) ---
        data['token_len'] = data['input'].apply(
            lambda x: len(tokenizer.encode(x, truncation=False))
        )

        print("Example input text:")
        display(data.input.iloc[0])

        print("\nToken length stats:")
        print(data['token_len'].describe())

        # How many rows exceed max_len?
        trunc_count = (data['token_len'] > max_len).sum()
        total = len(data)
        print(f"\nRows exceeding max_len={max_len}: {trunc_count} / {total} "
              f"({trunc_count / total:.2%})")

        # Optional: quick sanity check of a few long examples
        if trunc_count > 0:
            print("\nExample rows that will be truncated:")
            display(
                data.loc[data['token_len'] > max_len, ['token_len', 'input']]
                    .head(3)
            )

        # === continue with your embedding generation code below this ===
        # e.g., using a dataloader over data['input'] with truncation=True, max_length=max_len



Processing database: /content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/Production/data/fibvid_data.db

----- Mode: text_emo (max_len=500) -----

Raw data preview for /content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/Production/data/fibvid (text_emo):


,SPLIT,tweet_id,is_root,root_node,label,parent_id,text,target_emotion_stance,create_date,root_created,time_elapsed,0m,1m,15m,20m,60m,90m,24h,48h,gt_48h
0,train,1223618900044152834,1,1223618900044152834,1,1223618900044152834,"First Sen. Warren, & now Speaker Pelosi questi...",neutral emotion & AGAINST stance towards Warre...,2020-02-01T14:47:34.000Z,2020-02-01T14:47:34.000Z,0.000000,1,1,1,1,1,1,1,1,1
1,train,1223789646989144065,1,1223789646989144065,1,1223789646989144065,Statement on the final installment of the CNN/...,neutral emotion & NEUTRAL stance towards CNN;n...,2020-02-02T02:06:03.000Z,2020-02-02T02:06:03.000Z,0.000000,1,1,1,1,1,1,1,1,1
2,train,1223789928192057346,0,1223789646989144065,1,1223789646989144065,@NateSilver538,neutral emotion & FAVOR stance towards @NateSi...,2020-02-02T02:07:10.000Z,2020-02-02T02:06:03.000Z,1.116667,0,0,1,1,1,1,1,1,1
3,train,1223789991798636544,0,1223789646989144065,1,1223789646989144065,that respondents name was Shmoe Shmiden,neutral emotion & NEUTRAL stance towards Shmoe...,2020-02-02T02:07:25.000Z,2020-02-02T02:06:03.000Z,1.366667,0,0,1,1,1,1,1,1,1
4,train,1223790057133346816,0,1223789646989144065,1,1223789646989144065,Read: Bernie polled too highly and they looked...,neutral emotion & AGAINST stance towards Bernie,2020-02-02T02:07:41.000Z,2020-02-02T02:06:03.000Z,1.633333,0,0,1,1,1,1,1,1,1


Example raw text:


'First Sen. Warren, & now Speaker Pelosi question the legitimacy of Chief Justice Roberts & the Judicial Branch. This is all Democrats have. Their case fell apart so they will attack the legitimacy & credibility of everything in sight: the 2020 election, the Judiciary, & more.'

Token indices sequence length is longer than the specified maximum sequence length for this model (636 > 512). Running this sequence through the model will result in indexing errors


...

Example raw text:


'I might get an interview with Mike Huckabee about his weight loss/diabetes issue on Weds morning. Stay tuned!'

Example input text:


'I might get an interview with Mike Huckabee about his weight loss/diabetes issue on Weds morning. Stay tuned!'


Token length stats:
count    103672.000000
mean         58.406011
std          30.227253
min           6.000000
25%          39.000000
50%          58.000000
75%          72.000000
max         405.000000
Name: token_len, dtype: float64

Rows exceeding max_len=500: 0 / 103672 (0.00%)


In [ ]:
#checking if 500 max token is ok
import sqlite3
import pandas as pd
from transformers import AutoTokenizer


# =============================================================================
# Step 4: Set Up Dataset Paths and Configurations
# =============================================================================
base_path = '/content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/Production/data'

# Define dataset paths and their input mode configurations (each mode has a max length setting)
datasets = {
    f'{base_path}/fibvid': {'text_emo': 500, 'text_only': 500},
    f'{base_path}/ts':     {'text_emo': 500, 'text_only': 500},
}

# =============================================================================
# Step 5: Process Datasets and Generate BERT Embeddings into Separate Databases
# =============================================================================
for dataset_path, mode_config in datasets.items():
    db_path = f'{dataset_path}_data.db'
    print(f"\n==============================")
    print(f"Processing database: {db_path}")
    con = sqlite3.connect(db_path)

    for input_mode, max_len in mode_config.items():
        print(f"\n----- Mode: {input_mode} (max_len={max_len}) -----")

        # --- Load data deterministically ---
        data = pd.read_sql_query('SELECT * FROM data', con)

        print(f"\nRaw data preview for {dataset_path} ({input_mode}):")
        display(data.head())
        print("Example raw text:")
        display(data.text.iloc[0])

        # --- Build the input text used for BERT ---
        if input_mode == 'text_emo':
            # Emotion + stance + original text
            #data['input'] = data['target_emotion_stance'].fillna('') + '[SEP]' + data['text'].fillna('')
            data['tesa_tags'] = data['target_emotion_stance'].fillna("").apply(compact_tesa)
            data['input'] = data['tesa_tags'] + " [SEP] " + data['text'].fillna("")

        else:
            # Text only
            data['input'] = data['text'].fillna('')

        # --- NEW: compute token lengths for the whole dataset (no truncation) ---
        data['token_len'] = data['input'].apply(
            lambda x: len(tokenizer.encode(x, truncation=False))
        )

        print("Example input text:")
        display(data.input.iloc[0])

        print("\nToken length stats:")
        print(data['token_len'].describe())

        # How many rows exceed max_len?
        trunc_count = (data['token_len'] > max_len).sum()
        total = len(data)
        print(f"\nRows exceeding max_len={max_len}: {trunc_count} / {total} "
              f"({trunc_count / total:.2%})")

        # Optional: quick sanity check of a few long examples
        if trunc_count > 0:
            print("\nExample rows that will be truncated:")
            display(
                data.loc[data['token_len'] > max_len, ['token_len', 'input']]
                    .head(3)
            )

        # === continue with your embedding generation code below this ===
        # e.g., using a dataloader over data['input'] with truncation=True, max_length=max_len



Processing database: /content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/Production/data/fibvid_data.db

----- Mode: text_emo (max_len=500) -----

Raw data preview for /content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/Production/data/fibvid (text_emo):


,SPLIT,tweet_id,is_root,root_node,label,parent_id,text,target_emotion_stance,create_date,root_created,time_elapsed,0m,1m,15m,20m,60m,90m,24h,48h,gt_48h
0,train,1223618900044152834,1,1223618900044152834,1,1223618900044152834,"First Sen. Warren, & now Speaker Pelosi questi...",neutral emotion & AGAINST stance towards Warre...,2020-02-01T14:47:34.000Z,2020-02-01T14:47:34.000Z,0.000000,1,1,1,1,1,1,1,1,1
1,train,1223789646989144065,1,1223789646989144065,1,1223789646989144065,Statement on the final installment of the CNN/...,neutral emotion & NEUTRAL stance towards CNN;n...,2020-02-02T02:06:03.000Z,2020-02-02T02:06:03.000Z,0.000000,1,1,1,1,1,1,1,1,1
2,train,1223789928192057346,0,1223789646989144065,1,1223789646989144065,@NateSilver538,neutral emotion & FAVOR stance towards @NateSi...,2020-02-02T02:07:10.000Z,2020-02-02T02:06:03.000Z,1.116667,0,0,1,1,1,1,1,1,1
3,train,1223789991798636544,0,1223789646989144065,1,1223789646989144065,that respondents name was Shmoe Shmiden,neutral emotion & NEUTRAL stance towards Shmoe...,2020-02-02T02:07:25.000Z,2020-02-02T02:06:03.000Z,1.366667,0,0,1,1,1,1,1,1,1
4,train,1223790057133346816,0,1223789646989144065,1,1223789646989144065,Read: Bernie polled too highly and they looked...,neutral emotion & AGAINST stance towards Bernie,2020-02-02T02:07:41.000Z,2020-02-02T02:06:03.000Z,1.633333,0,0,1,1,1,1,1,1,1


Example raw text:


'First Sen. Warren, & now Speaker Pelosi question the legitimacy of Chief Justice Roberts & the Judicial Branch. This is all Democrats have. Their case fell apart so they will attack the legitimacy & credibility of everything in sight: the 2020 election, the Judiciary, & more.'

Example input text:


...

Example raw text:


'I might get an interview with Mike Huckabee about his weight loss/diabetes issue on Weds morning. Stay tuned!'

Example input text:


'I might get an interview with Mike Huckabee about his weight loss/diabetes issue on Weds morning. Stay tuned!'


Token length stats:
count    103672.000000
mean         58.406011
std          30.227253
min           6.000000
25%          39.000000
50%          58.000000
75%          72.000000
max         405.000000
Name: token_len, dtype: float64

Rows exceeding max_len=500: 0 / 103672 (0.00%)


In [ ]:
# =============================================================================
# Step 4: Set Up Dataset Paths and Configurations
# =============================================================================
base_path = '/content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/Production/data'

# Define dataset paths and their input mode configurations (each mode has a max length setting)
datasets = {
    f'{base_path}/fibvid': {'text_emo': 500, 'text_only': 500},
    f'{base_path}/ts':     {'text_emo': 500, 'text_only': 500},
}

# =============================================================================
# Step 5: Process Datasets and Generate BERT Embeddings into Separate Databases
# =============================================================================
for dataset_path, mode_config in datasets.items():
    db_path = f'{dataset_path}_data.db'
    print(f"Processing database: {db_path}")
    con = sqlite3.connect(db_path)

    for input_mode, max_len in mode_config.items():
        # --- Load data deterministically ---
        data = pd.read_sql_query('SELECT * FROM data', con)

        print(f"\nRaw data preview for {dataset_path} ({input_mode}):")
        display(data.head())
        print("Example raw text:")
        display(data.text.iloc[0])

        # --- Build the input text used for BERT ---
        if input_mode == 'text_emo':
            # Emotion + stance + original text
            #data['input'] = data['target_emotion_stance'].fillna('') + '[SEP]' + data['text'].fillna('')
            data['tesa_tags'] = data['target_emotion_stance'].fillna("").apply(compact_tesa)
            data['input'] = data['tesa_tags'] + " [SEP] " + data['text'].fillna("")



        else:
            # Text only
            data['input'] = data['text']

        print("Example constructed input:")
        display(data.input.iloc[0])

        # --- Encode using deterministic BERT encoder ---
        print(f"\nEncoding {len(data)} rows for mode: {input_mode} (max_len={max_len})")
        embeddings = bertEncode(
            data['input'].tolist(),
            max_len=max_len,
            batch_size=32,
        )
        # embeddings is a list of numpy arrays
        data['bert_embs'] = embeddings

        # Keep only tweet_id + embeddings and JSON-encode for SQLite
        processed_data = data[['tweet_id', 'bert_embs']].copy()
        processed_data['bert_embs'] = processed_data['bert_embs'].progress_apply(
            lambda x: json.dumps(x.tolist() if isinstance(x, np.ndarray) else x)
        )

        print("\nProcessed data sample:")
        display(processed_data.head())

        embeddings_db_path = f'{dataset_path}_bertlarge_embeddings_{input_mode}_timelock{SEED}.db'
        print(f"\nSaving embeddings to: {embeddings_db_path} (table: {input_mode})")

        out_con = sqlite3.connect(embeddings_db_path)
        processed_data.to_sql(input_mode, out_con, index=False, if_exists='replace')
        out_con.close()

    con.close()


Processing database: /content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/Production/data/fibvid_data.db

Raw data preview for /content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/Production/data/fibvid (text_emo):


,SPLIT,tweet_id,is_root,root_node,label,parent_id,text,target_emotion_stance,create_date,root_created,time_elapsed,0m,1m,15m,20m,60m,90m,24h,48h,gt_48h
0,train,1223618900044152834,1,1223618900044152834,1,1223618900044152834,"First Sen. Warren, & now Speaker Pelosi questi...",neutral emotion & AGAINST stance towards Warre...,2020-02-01T14:47:34.000Z,2020-02-01T14:47:34.000Z,0.000000,1,1,1,1,1,1,1,1,1
1,train,1223789646989144065,1,1223789646989144065,1,1223789646989144065,Statement on the final installment of the CNN/...,neutral emotion & NEUTRAL stance towards CNN;n...,2020-02-02T02:06:03.000Z,2020-02-02T02:06:03.000Z,0.000000,1,1,1,1,1,1,1,1,1
2,train,1223789928192057346,0,1223789646989144065,1,1223789646989144065,@NateSilver538,neutral emotion & FAVOR stance towards @NateSi...,2020-02-02T02:07:10.000Z,2020-02-02T02:06:03.000Z,1.116667,0,0,1,1,1,1,1,1,1
3,train,1223789991798636544,0,1223789646989144065,1,1223789646989144065,that respondents name was Shmoe Shmiden,neutral emotion & NEUTRAL stance towards Shmoe...,2020-02-02T02:07:25.000Z,2020-02-02T02:06:03.000Z,1.366667,0,0,1,1,1,1,1,1,1
4,train,1223790057133346816,0,1223789646989144065,1,1223789646989144065,Read: Bernie polled too highly and they looked...,neutral emotion & AGAINST stance towards Bernie,2020-02-02T02:07:41.000Z,2020-02-02T02:06:03.000Z,1.633333,0,0,1,1,1,1,1,1,1


Example raw text:


'First Sen. Warren, & now Speaker Pelosi question the legitimacy of Chief Justice Roberts & the Judicial Branch. This is all Democrats have. Their case fell apart so they will attack the legitimacy & credibility of everything in sight: the 2020 election, the Judiciary, & more.'

Example constructed input:


...


Encoding 103672 rows for mode: text_only (max_len=500)


100%|██████████| 103672/103672 [01:18<00:00, 1328.43it/s]



Processed data sample:


,tweet_id,bert_embs
0,46516,"[-0.2708462178707123, -0.6541179418563843, 0.7..."
1,378,"[-0.08169658482074738, -0.4504814147949219, 0...."
2,46541,"[-0.2693890333175659, -0.6346859931945801, 0.7..."
3,88,"[-0.037797488272190094, -0.27148136496543884, ..."
4,13397,"[-0.24520893394947052, -0.728566586971283, 0.7..."



Saving embeddings to: /content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/Production/data/ts_bertlarge_embeddings_text_only_timelock9898.db (table: text_only)


In [ ]:
#takes 4hrs + to process